# 연합뉴스 전처리 

In [12]:
import pandas as pd
import re

df = pd.read_csv("naver(연합뉴스).csv", parse_dates=['time'])
texts = df.loc[:, 'body']

In [14]:
pre_texts = []

for text in texts:
    cleaned_text = re.sub('연합뉴스', '', str(text))
    cleaned_text = re.sub('\[.*=', '', str(text))
    cleaned_text = re.sub('[a-zA-z0-9]+@.+', '', cleaned_text)
    cleaned_text = re.sub('(\', \')', '', cleaned_text)
    cleaned_text = re.sub('(\\\\r   )', '', cleaned_text)
    cleaned_text = re.sub("(\\\\'|\\')", '', cleaned_text)
    cleaned_text = re.sub('(  |    |  )',' ', cleaned_text)
    cleaned_text = re.sub("[☎※:\-◇▲◆,■’`“”\"]", '', cleaned_text)
    cleaned_text = re.sub("(\\\\n)", '', cleaned_text)
    cleaned_text = re.sub("(\\\\t)", '', cleaned_text)
    cleaned_text = re.sub("[0-9](\.)[0-9]", '', cleaned_text) #다시
    cleaned_text = re.sub("[%!-]", '', cleaned_text)
    
    cleaned_text = re.sub("[ㆍ…→\(\)\[\]]", ' ', cleaned_text)
    cleaned_text = re.sub('(https?.+)?[A-Za-z0-9]{1,}\.[A-Za-z0-9]{1,10}\.?[A-Za-z]{1,}\.?[A-Za-z]{1,}', '', cleaned_text)
    
    if len(re.findall('[가-힣]', cleaned_text)) < 30 :
        cleaned_text = ''
        
    pre_texts.append(cleaned_text.strip())

In [15]:
df['body']= pre_texts

In [16]:
idx_del = df[df['body'] == ''].index
df = df.drop(idx_del)

In [17]:
idx_del

Int64Index([   55,    63,   111,   112,   116,   123,   141,   153,   161,
              178,
            ...
            73676, 73697, 73716, 73722, 73724, 73726, 73731, 73736, 73737,
            73762],
           dtype='int64', length=3901)

In [21]:
import pandas as pd
from tqdm import tqdm
from ekonlpy.sentiment import MPCK

mpck = MPCK()
# 기사 문장별 split 후 ngramize

ngram_list=[]
for i in tqdm(range(len(df['body']))):
    try:
        sent_ngram=[]
        sent=df['body'][i].split('.')
        
        for s in sent:
            ngrams = mpck.ngramize(mpck.tokenize(s), keep_overlapping_ngram=False)
            
            if ngrams:
                sent_ngram.append(ngrams)
                
        ngram_list.append(sent_ngram)
        
    except:
        print(i)
        
df['ngram']=ngram_list

100%|████████████████████████████████████████████████████████████████████████████| 73775/73775 [45:10<00:00, 27.22it/s]


In [24]:
df.to_csv('연합뉴스_전처리.csv', encoding='utf-8')